In [1]:
df_c = pd.read_hdf('00_SAVIC_Examples/SAVIC_Examples.h5', key = 'SAVIC-Q_C_post_Q')
df_c

,beta_par_core,alph_c,unstable,Pow_core,kB_angle
0,1.0,1.0,False,NaN,NaN
1,0.5,3.2,True,0.171887,0.001113
2,1.0,0.4,True,0.000287,0.002231
3,12.0,1.2,True,0.001800,0.001649


In [2]:
df_c.to_hdf('00_SAVIC_Examples/SAVIC_Examples.h5', key = 'SAVIC-Q_C_post_Q')

In [3]:
from sklearn.mixture import GaussianMixture

## SAVIC-C - C

### load SAVIC-C - C GM Classifier

In [4]:
gmm_c_name = 'Output/ML/models/GMM_C/GMM_C'

# reload
means = np.load(gmm_c_name + '_means.npy')
covar = np.load(gmm_c_name + '_covariances.npy')
gmm_c = GaussianMixture(n_components = len(means), covariance_type='full')
gmm_c.precisions_cholesky_ = np.linalg.cholesky(np.linalg.inv(covar))
gmm_c.weights_ = np.load(gmm_c_name + '_weights.npy')
gmm_c.means_ = means
gmm_c.covariances_ = covar



##### generate example data

In [5]:
df_c = pd.read_hdf('00_SAVIC_Examples/SAVIC_Examples.h5', key = 'SAVIC-Q_C_post_Q')
df_c

,beta_par_core,alph_c,unstable,Pow_core,kB_angle
0,1.0,1.0,False,NaN,NaN
1,0.5,3.2,True,0.171887,0.001113
2,1.0,0.4,True,0.000287,0.002231
3,12.0,1.2,True,0.001800,0.001649


##### filter and format the input

In [6]:
df_c_uns = df_c[df_c['unstable']].drop(columns = ['unstable'])
df_c_uns.drop(columns = ['Pow_core'], inplace = True)
df_c_uns

,beta_par_core,alph_c,kB_angle
1,0.5,3.2,0.001113
2,1.0,0.4,0.002231
3,12.0,1.2,0.001649


##### add instability thresholds

In [7]:
df_c_uns_pre_SC = df_c_uns.copy()
df_c_uns_pre_SC['uns_IC'] = (1. + 0.367 / ( df_c_uns_pre_SC['beta_par_core'] - 0.011 )**0.364) < df_c_uns_pre_SC['alph_c']
df_c_uns_pre_SC['uns_Mirror'] = (1. + 0.702 / ( df_c_uns_pre_SC['beta_par_core'] + 0.009 )**0.674) < df_c_uns_pre_SC['alph_c']
df_c_uns_pre_SC['uns_FM'] = (1. - 0.408 / ( df_c_uns_pre_SC['beta_par_core'] - 0.410 )**0.529) > df_c_uns_pre_SC['alph_c']
df_c_uns_pre_SC['uns_OFH'] = (1. - 1.454 / ( df_c_uns_pre_SC['beta_par_core'] + 0.178 )**1.023) > df_c_uns_pre_SC['alph_c']
df_c_uns_pre_SC['uns_core'] = df_c_uns_pre_SC['uns_OFH'] | df_c_uns_pre_SC['uns_FM'] | df_c_uns_pre_SC['uns_Mirror'] | df_c_uns_pre_SC['uns_IC']
df_c_uns_pre_SC.drop(columns = ['uns_Mirror'], inplace = True)
df_c_uns_pre_SC

,beta_par_core,alph_c,kB_angle,uns_IC,uns_FM,uns_OFH,uns_core
1,0.5,3.2,0.001113,True,False,False,True
2,1.0,0.4,0.002231,False,True,False,True
3,12.0,1.2,0.001649,True,False,False,True


In [8]:
df_c_uns_pre_SC[['beta_par_core', 'alph_c']] = np.log10(df_c_uns_pre_SC[['beta_par_core', 'alph_c']])
df_c_uns_pre_SC.rename(columns={"beta_par_core": "log_beta_par_core", "alph_c": "log_alph_c"}, inplace = True)
df_c_uns_pre_SC

,log_beta_par_core,log_alph_c,kB_angle,uns_IC,uns_FM,uns_OFH,uns_core
1,-0.301030,0.505150,0.001113,True,False,False,True
2,0.000000,-0.397940,0.002231,False,True,False,True
3,1.079181,0.079181,0.001649,True,False,False,True


##### run data through classifier

In [9]:
df_c_uns_post_SC = df_c_uns_pre_SC.copy()
df_c_uns_post_SC['cluster'] = gmm_c.predict(df_c_uns_pre_SC[['log_beta_par_core', 'log_alph_c', 'kB_angle', \
                                                             'uns_IC', 'uns_FM', 'uns_OFH', 'uns_core']].values)
df_c_uns_post_SC

,log_beta_par_core,log_alph_c,kB_angle,uns_IC,uns_FM,uns_OFH,uns_core,cluster
1,-0.301030,0.505150,0.001113,True,False,False,True,0
2,0.000000,-0.397940,0.002231,False,True,False,True,1
3,1.079181,0.079181,0.001649,True,False,False,True,0


##### revert to input data values

In [10]:
df_c_uns_post_SC[['log_beta_par_core', 'log_alph_c']] = pow(10., df_c_uns_post_SC[['log_beta_par_core', 'log_alph_c']])
df_c_uns_post_SC.rename(columns={"log_beta_par_core": "beta_par_core", "log_alph_c": "alph_c"}, inplace = True)
df_c_uns_post_SC

,beta_par_core,alph_c,kB_angle,uns_IC,uns_FM,uns_OFH,uns_core,cluster
1,0.5,3.2,0.001113,True,False,False,True,0
2,1.0,0.4,0.002231,False,True,False,True,1
3,12.0,1.2,0.001649,True,False,False,True,0


##### add instability labels

In [11]:
df_c_uns_post_SC['ins_type'] = np.nan
ins_types_c = ['Ion Cyclotron', 'Parallel Firehose', 'Mirror', 'Oblique Firehose']
for cluster, ins_type in zip(np.arange(4), ins_types_c):
    df_c_uns_post_SC['ins_type'][df_c_uns_post_SC['cluster'] == cluster] = ins_type
df_c_uns_post_SC

,beta_par_core,alph_c,kB_angle,uns_IC,uns_FM,uns_OFH,uns_core,cluster,ins_type
1,0.5,3.2,0.001113,True,False,False,True,0,Ion Cyclotron
2,1.0,0.4,0.002231,False,True,False,True,1,Parallel Firehose
3,12.0,1.2,0.001649,True,False,False,True,0,Ion Cyclotron


##### restore initial input format

In [12]:
df_c_post_SC = df_c.copy()
df_c_post_SC['ins_type'] = np.nan
df_c_post_SC.loc[df_c_uns_post_SC.index, ['ins_type']] = df_c_uns_post_SC['ins_type']
df_c_post_SC

,beta_par_core,alph_c,unstable,Pow_core,kB_angle,ins_type
0,1.0,1.0,False,NaN,NaN,NaN
1,0.5,3.2,True,0.171887,0.001113,Ion Cyclotron
2,1.0,0.4,True,0.000287,0.002231,Parallel Firehose
3,12.0,1.2,True,0.001800,0.001649,Ion Cyclotron


### load SAVIC-C - C processing function

In [13]:
def SAVIC_C_C(df_c_):
    
    df_c_uns_ = df_c_[df_c_['unstable']].drop(columns = ['unstable'])
    df_c_uns_.drop(columns = ['Pow_core'], inplace = True)
    
    df_c_uns_pre_SC_ = df_c_uns_.copy()
    df_c_uns_pre_SC_['uns_IC'] = (1. + 0.367 / ( df_c_uns_pre_SC_['beta_par_core'] - 0.011 )**0.364) < df_c_uns_pre_SC_['alph_c']
    df_c_uns_pre_SC_['uns_Mirror'] = (1. + 0.702 / ( df_c_uns_pre_SC_['beta_par_core'] + 0.009 )**0.674) < df_c_uns_pre_SC_['alph_c']
    df_c_uns_pre_SC_['uns_FM'] = (1. - 0.408 / ( df_c_uns_pre_SC_['beta_par_core'] - 0.410 )**0.529) > df_c_uns_pre_SC_['alph_c']
    df_c_uns_pre_SC_['uns_OFH'] = (1. - 1.454 / ( df_c_uns_pre_SC_['beta_par_core'] + 0.178 )**1.023) > df_c_uns_pre_SC_['alph_c']
    df_c_uns_pre_SC_['uns_core'] = df_c_uns_pre_SC_['uns_OFH'] | df_c_uns_pre_SC_['uns_FM'] | df_c_uns_pre_SC_['uns_Mirror'] | df_c_uns_pre_SC_['uns_IC']
    df_c_uns_pre_SC_.drop(columns = ['uns_Mirror'], inplace = True)
    
    df_c_uns_pre_SC_[['beta_par_core', 'alph_c']] = np.log10(df_c_uns_pre_SC_[['beta_par_core', 'alph_c']])
    df_c_uns_pre_SC_.rename(columns={"beta_par_core": "log_beta_par_core", "alph_c": "log_alph_c"}, inplace = True)
    
    df_c_uns_post_SC_ = df_c_uns_pre_SC_.copy()
    df_c_uns_post_SC_['cluster'] = gmm_c.predict(df_c_uns_pre_SC_[['log_beta_par_core', 'log_alph_c', 'kB_angle', \
                                                                   'uns_IC', 'uns_FM', 'uns_OFH', 'uns_core']].values)
    
    df_c_uns_post_SC_[['log_beta_par_core', 'log_alph_c']] = \
    pow(10., df_c_uns_post_SC_[['log_beta_par_core', 'log_alph_c']])
    df_c_uns_post_SC_.rename(columns={"log_beta_par_core": "beta_par_core", "log_alph_c": "alph_c"}, inplace = True)
    
    df_c_uns_post_SC_['ins_type'] = np.nan
    ins_types = ['Ion Cyclotron', 'Parallel Firehose', 'Mirror', 'Oblique Firehose']
    for cluster, ins_type in zip(np.arange(4), ins_types):
        df_c_uns_post_SC_['ins_type'][df_c_uns_post_SC_['cluster'] == cluster] = ins_type
    
    df_c_post_SC_ = df_c_.copy()
    df_c_post_SC_['ins_type'] = np.nan
    df_c_post_SC_.loc[df_c_uns_post_SC_.index, ['ins_type']] = df_c_uns_post_SC_['ins_type']
    
    return df_c_post_SC_
    

### run SAVIC-C - C 

In [14]:
df_c_uns_post_SC = SAVIC_C_C(pd.read_hdf('00_SAVIC_Examples/SAVIC_Examples.h5', key = 'SAVIC-Q_C_post_Q'))
df_c_uns_post_SC

,beta_par_core,alph_c,unstable,Pow_core,kB_angle,ins_type
0,1.0,1.0,False,NaN,NaN,NaN
1,0.5,3.2,True,0.171887,0.001113,Ion Cyclotron
2,1.0,0.4,True,0.000287,0.002231,Parallel Firehose
3,12.0,1.2,True,0.001800,0.001649,Ion Cyclotron


##### save results

In [15]:
#df_c_uns_post_SC.to_hdf('00_SAVIC_Examples/SAVIC_Examples.h5', key = 'SAVIC-C_C_post_C')

## SAVIC-C - CB

### load SAVIC-C - CB GM Classifier

In [16]:
gmm_cb_name = 'Output/ML/models/GMM_CB/GMM_CB'
means = np.load(gmm_cb_name + '_means.npy')
covar = np.load(gmm_cb_name + '_covariances.npy')
gmm_cb = GaussianMixture(n_components = len(means), covariance_type='full')
gmm_cb.precisions_cholesky_ = np.linalg.cholesky(np.linalg.inv(covar))
gmm_cb.weights_ = np.load(gmm_cb_name + '_weights.npy')
gmm_cb.means_ = means
gmm_cb.covariances_ = covar


##### generate example data

In [17]:
df_cb = pd.read_hdf('00_SAVIC_Examples/SAVIC_Examples.h5', key = 'SAVIC-Q_CB_post_Q')
df_cb

,beta_par_core,alph_c,tau_b,alph_b,D_b,vv_b,unstable,group,Pow_core,Pow_beam,kB_angle
0,1.0,1.0,1.0,1.0,0.05,0.5,False,NaN,NaN,NaN,NaN
1,1.5,2.5,0.8,1.0,0.05,0.5,True,3.0,0.193271,0.000000,0.004137
2,0.5,1.0,1.0,3.5,0.10,1.5,True,5.0,0.000000,0.123028,0.003983
3,0.8,1.1,1.0,1.2,0.05,1.8,True,5.0,0.000000,0.004923,0.001414
4,0.5,0.7,0.8,0.8,0.01,0.2,False,NaN,NaN,NaN,NaN
5,0.8,3.1,1.0,3.9,0.10,1.9,True,1.0,0.214777,0.000728,0.000991


In [18]:
df_cb_uns = df_cb[df_cb['unstable']].drop(columns = ['unstable'])
df_cb_uns

,beta_par_core,alph_c,tau_b,alph_b,D_b,vv_b,group,Pow_core,Pow_beam,kB_angle
1,1.5,2.5,0.8,1.0,0.05,0.5,3.0,0.193271,0.000000,0.004137
2,0.5,1.0,1.0,3.5,0.10,1.5,5.0,0.000000,0.123028,0.003983
3,0.8,1.1,1.0,1.2,0.05,1.8,5.0,0.000000,0.004923,0.001414
5,0.8,3.1,1.0,3.9,0.10,1.9,1.0,0.214777,0.000728,0.000991


##### add instability thresholds

In [19]:
df_cb_uns_pre_SC = df_cb_uns.copy()

df_cb_uns_pre_SC['uns_IC'] = (1. + 0.367 / ( df_cb_uns_pre_SC['beta_par_core'] - 0.011 )**0.364) < df_cb_uns_pre_SC['alph_c']
df_cb_uns_pre_SC['uns_Mirror'] = (1. + 0.702 / ( df_cb_uns_pre_SC['beta_par_core'] + 0.009 )**0.674) < df_cb_uns_pre_SC['alph_c']
df_cb_uns_pre_SC['uns_FM'] = (1. - 0.408 / ( df_cb_uns_pre_SC['beta_par_core'] - 0.410 )**0.529) > df_cb_uns_pre_SC['alph_c']
df_cb_uns_pre_SC['uns_OFH'] = (1. - 1.454 / ( df_cb_uns_pre_SC['beta_par_core'] + 0.178 )**1.023) > df_cb_uns_pre_SC['alph_c']
df_cb_uns_pre_SC['uns_core'] = df_cb_uns_pre_SC['uns_OFH'] | df_cb_uns_pre_SC['uns_FM'] | df_cb_uns_pre_SC['uns_Mirror'] | df_cb_uns_pre_SC['uns_IC']

df_cb_uns_pre_SC['beta_par_beam'] = df_cb_uns_pre_SC['beta_par_core'] * df_cb_uns_pre_SC['D_b'] / df_cb_uns_pre_SC['tau_b']
df_cb_uns_pre_SC['lambda_FH_beta'] = ( (df_cb_uns_pre_SC['beta_par_core'] + df_cb_uns_pre_SC['beta_par_beam']) - \
(df_cb_uns_pre_SC['beta_par_core'] * df_cb_uns_pre_SC['alph_c'] + df_cb_uns_pre_SC['beta_par_beam'] * df_cb_uns_pre_SC['alph_b']) ) / 2.
df_cb_uns_pre_SC['lambda_FH_beam'] = df_cb_uns_pre_SC['D_b'] * df_cb_uns_pre_SC['vv_b']**2.
df_cb_uns_pre_SC['lambda_FH'] = df_cb_uns_pre_SC['lambda_FH_beta'] + df_cb_uns_pre_SC['lambda_FH_beam']# + df_cb_uns_pre_SC['lambda_FH_alpha']
df_cb_uns_pre_SC['lambda_FH'] = df_cb_uns_pre_SC['lambda_FH'] > 1.

df_cb_uns_pre_SC['Pow_core'][df_cb_uns_pre_SC['Pow_core'] < 0] = 0
df_cb_uns_pre_SC['Pow_beam'][df_cb_uns_pre_SC['Pow_beam'] < 0] = 0

df_cb_uns_pre_SC

,beta_par_core,alph_c,tau_b,alph_b,D_b,vv_b,group,Pow_core,Pow_beam,kB_angle,uns_IC,uns_Mirror,uns_FM,uns_OFH,uns_core,beta_par_beam,lambda_FH_beta,lambda_FH_beam,lambda_FH
1,1.5,2.5,0.8,1.0,0.05,0.5,3.0,0.193271,0.000000,0.004137,True,True,False,False,True,0.09375,-1.1250,0.0125,False
2,0.5,1.0,1.0,3.5,0.10,1.5,5.0,0.000000,0.123028,0.003983,False,False,False,False,False,0.05000,-0.0625,0.2250,False
3,0.8,1.1,1.0,1.2,0.05,1.8,5.0,0.000000,0.004923,0.001414,False,False,False,False,False,0.04000,-0.0440,0.1620,False
5,0.8,3.1,1.0,3.9,0.10,1.9,1.0,0.214777,0.000728,0.000991,True,True,False,False,True,0.08000,-0.9560,0.3610,False


##### filter and format the input

In [20]:
column_list_cb = ['beta_par_core', 'alph_c', 'kB_angle', \
                  'tau_b', 'D_b', 'alph_b', 'vv_b', 'Pow_core', 'Pow_beam', \
                  'uns_IC', 'uns_FM', 'uns_OFH', 'uns_core', 'lambda_FH']

df_cb_uns_pre_SC_cut = df_cb_uns_pre_SC[column_list_cb]
df_cb_uns_pre_SC_cut

,beta_par_core,alph_c,kB_angle,tau_b,D_b,alph_b,vv_b,Pow_core,Pow_beam,uns_IC,uns_FM,uns_OFH,uns_core,lambda_FH
1,1.5,2.5,0.004137,0.8,0.05,1.0,0.5,0.193271,0.000000,True,False,False,True,False
2,0.5,1.0,0.003983,1.0,0.10,3.5,1.5,0.000000,0.123028,False,False,False,False,False
3,0.8,1.1,0.001414,1.0,0.05,1.2,1.8,0.000000,0.004923,False,False,False,False,False
5,0.8,3.1,0.000991,1.0,0.10,3.9,1.9,0.214777,0.000728,True,False,False,True,False


In [21]:
df_cb_uns_pre_SC_cut[['beta_par_core', 'alph_c']] = np.log10(df_cb_uns_pre_SC_cut[['beta_par_core', 'alph_c']])
df_cb_uns_pre_SC_cut.rename(columns={"beta_par_core": "log_beta_par_core", "alph_c": "log_alph_c"}, inplace = True)
df_cb_uns_pre_SC_cut

,log_beta_par_core,log_alph_c,kB_angle,tau_b,D_b,alph_b,vv_b,Pow_core,Pow_beam,uns_IC,uns_FM,uns_OFH,uns_core,lambda_FH
1,0.176091,0.397940,0.004137,0.8,0.05,1.0,0.5,0.193271,0.000000,True,False,False,True,False
2,-0.301030,0.000000,0.003983,1.0,0.10,3.5,1.5,0.000000,0.123028,False,False,False,False,False
3,-0.096910,0.041393,0.001414,1.0,0.05,1.2,1.8,0.000000,0.004923,False,False,False,False,False
5,-0.096910,0.491362,0.000991,1.0,0.10,3.9,1.9,0.214777,0.000728,True,False,False,True,False


##### run data through classifier

In [22]:
df_cb_uns_post_SC = df_cb_uns_pre_SC_cut.copy()
df_cb_uns_post_SC['cluster'] = gmm_cb.predict(df_cb_uns_pre_SC_cut.values)
df_cb_uns_post_SC

,log_beta_par_core,log_alph_c,kB_angle,tau_b,D_b,alph_b,vv_b,Pow_core,Pow_beam,uns_IC,uns_FM,uns_OFH,uns_core,lambda_FH,cluster
1,0.176091,0.397940,0.004137,0.8,0.05,1.0,0.5,0.193271,0.000000,True,False,False,True,False,6
2,-0.301030,0.000000,0.003983,1.0,0.10,3.5,1.5,0.000000,0.123028,False,False,False,False,False,4
3,-0.096910,0.041393,0.001414,1.0,0.05,1.2,1.8,0.000000,0.004923,False,False,False,False,False,4
5,-0.096910,0.491362,0.000991,1.0,0.10,3.9,1.9,0.214777,0.000728,True,False,False,True,False,6


##### revert to input data values

In [23]:
df_cb_uns_post_SC[['log_beta_par_core', 'log_alph_c']] = pow(10., df_cb_uns_post_SC[['log_beta_par_core', 'log_alph_c']])
df_cb_uns_post_SC.rename(columns={"log_beta_par_core": "beta_par_core", "log_alph_c": "alph_c"}, inplace = True)
df_cb_uns_post_SC

,beta_par_core,alph_c,kB_angle,tau_b,D_b,alph_b,vv_b,Pow_core,Pow_beam,uns_IC,uns_FM,uns_OFH,uns_core,lambda_FH,cluster
1,1.5,2.5,0.004137,0.8,0.05,1.0,0.5,0.193271,0.000000,True,False,False,True,False,6
2,0.5,1.0,0.003983,1.0,0.10,3.5,1.5,0.000000,0.123028,False,False,False,False,False,4
3,0.8,1.1,0.001414,1.0,0.05,1.2,1.8,0.000000,0.004923,False,False,False,False,False,4
5,0.8,3.1,0.000991,1.0,0.10,3.9,1.9,0.214777,0.000728,True,False,False,True,False,6


##### add instability labels

In [24]:
df_cb_uns_post_SC['ins_type'] = np.nan
ins_types_cb = ['Oblique FM (B); resonant with Core', 'IC (B); $T_\perp/T_{||} < 1$', 'Oblique FM (B)', \
                'Oblique Firehose', 'IC (B); $T_\perp/T_{||} > 1$', 'Parallel Firehose', 'IC (B), unstable core', 'IC (C)']
for cluster, ins_type in zip(np.arange(8), ins_types_cb):
    df_cb_uns_post_SC['ins_type'][df_cb_uns_post_SC['cluster'] == cluster] = ins_type
df_cb_uns_post_SC

,beta_par_core,alph_c,kB_angle,tau_b,D_b,alph_b,vv_b,Pow_core,Pow_beam,uns_IC,uns_FM,uns_OFH,uns_core,lambda_FH,cluster,ins_type
1,1.5,2.5,0.004137,0.8,0.05,1.0,0.5,0.193271,0.000000,True,False,False,True,False,6,"IC (B), unstable core"
2,0.5,1.0,0.003983,1.0,0.10,3.5,1.5,0.000000,0.123028,False,False,False,False,False,4,IC (B); $T_\perp/T_{||} > 1$
3,0.8,1.1,0.001414,1.0,0.05,1.2,1.8,0.000000,0.004923,False,False,False,False,False,4,IC (B); $T_\perp/T_{||} > 1$
5,0.8,3.1,0.000991,1.0,0.10,3.9,1.9,0.214777,0.000728,True,False,False,True,False,6,"IC (B), unstable core"


##### restore initial input format

In [25]:
df_cb_post_SC = df_cb.copy()
df_cb_post_SC['ins_type'] = np.nan
df_cb_post_SC.loc[df_cb_uns_post_SC.index, ['ins_type']] = df_cb_uns_post_SC['ins_type']
df_cb_post_SC

,beta_par_core,alph_c,tau_b,alph_b,D_b,vv_b,unstable,group,Pow_core,Pow_beam,kB_angle,ins_type
0,1.0,1.0,1.0,1.0,0.05,0.5,False,NaN,NaN,NaN,NaN,NaN
1,1.5,2.5,0.8,1.0,0.05,0.5,True,3.0,0.193271,0.000000,0.004137,"IC (B), unstable core"
2,0.5,1.0,1.0,3.5,0.10,1.5,True,5.0,0.000000,0.123028,0.003983,IC (B); $T_\perp/T_{||} > 1$
3,0.8,1.1,1.0,1.2,0.05,1.8,True,5.0,0.000000,0.004923,0.001414,IC (B); $T_\perp/T_{||} > 1$
4,0.5,0.7,0.8,0.8,0.01,0.2,False,NaN,NaN,NaN,NaN,NaN
5,0.8,3.1,1.0,3.9,0.10,1.9,True,1.0,0.214777,0.000728,0.000991,"IC (B), unstable core"


### load SAVIC-C - CB processing function

In [26]:
def SAVIC_C_CB(df_cb_):
    
    df_cb_uns_ = df_cb_[df_cb_['unstable']].drop(columns = ['unstable'])
    df_cb_uns_pre_SC_ = df_cb_uns_.copy()

    df_cb_uns_pre_SC_['uns_IC'] = (1. + 0.367 / ( df_cb_uns_pre_SC_['beta_par_core'] - 0.011 )**0.364) < df_cb_uns_pre_SC_['alph_c']
    df_cb_uns_pre_SC_['uns_Mirror'] = (1. + 0.702 / ( df_cb_uns_pre_SC_['beta_par_core'] + 0.009 )**0.674) < df_cb_uns_pre_SC_['alph_c']
    df_cb_uns_pre_SC_['uns_FM'] = (1. - 0.408 / ( df_cb_uns_pre_SC_['beta_par_core'] - 0.410 )**0.529) > df_cb_uns_pre_SC_['alph_c']
    df_cb_uns_pre_SC_['uns_OFH'] = (1. - 1.454 / ( df_cb_uns_pre_SC_['beta_par_core'] + 0.178 )**1.023) > df_cb_uns_pre_SC_['alph_c']
    df_cb_uns_pre_SC_['uns_core'] = df_cb_uns_pre_SC_['uns_OFH'] | df_cb_uns_pre_SC_['uns_FM'] | df_cb_uns_pre_SC_['uns_Mirror'] | df_cb_uns_pre_SC_['uns_IC']

    df_cb_uns_pre_SC_['beta_par_beam'] = df_cb_uns_pre_SC_['beta_par_core'] * df_cb_uns_pre_SC_['D_b'] / df_cb_uns_pre_SC_['tau_b']
    df_cb_uns_pre_SC_['lambda_FH_beta'] = ( (df_cb_uns_pre_SC_['beta_par_core'] + df_cb_uns_pre_SC_['beta_par_beam']) - \
    (df_cb_uns_pre_SC_['beta_par_core'] * df_cb_uns_pre_SC_['alph_c'] + df_cb_uns_pre_SC_['beta_par_beam'] * df_cb_uns_pre_SC_['alph_b']) ) / 2.
    df_cb_uns_pre_SC_['lambda_FH_beam'] = df_cb_uns_pre_SC_['D_b'] * df_cb_uns_pre_SC_['vv_b']**2.
    df_cb_uns_pre_SC_['lambda_FH'] = df_cb_uns_pre_SC_['lambda_FH_beta'] + df_cb_uns_pre_SC_['lambda_FH_beam']# + df_cb_uns_pre_SC_['lambda_FH_alpha']
    df_cb_uns_pre_SC_['lambda_FH'] = df_cb_uns_pre_SC_['lambda_FH'] > 1.

    df_cb_uns_pre_SC_['Pow_core'][df_cb_uns_pre_SC_['Pow_core'] < 0] = 0
    df_cb_uns_pre_SC_['Pow_beam'][df_cb_uns_pre_SC_['Pow_beam'] < 0] = 0

    
    column_list_cb_ = ['beta_par_core', 'alph_c', 'kB_angle', \
                       'tau_b', 'D_b', 'alph_b', 'vv_b', 'Pow_core', 'Pow_beam', \
                       'uns_IC', 'uns_FM', 'uns_OFH', 'uns_core', 'lambda_FH']

    df_cb_uns_pre_SC_cut_ = df_cb_uns_pre_SC_[column_list_cb_]
    
    df_cb_uns_pre_SC_cut_[['beta_par_core', 'alph_c']] = np.log10(df_cb_uns_pre_SC_cut_[['beta_par_core', 'alph_c']])
    df_cb_uns_pre_SC_cut_.rename(columns={"beta_par_core": "log_beta_par_core", "alph_c": "log_alph_c"}, inplace = True)
    
    df_cb_uns_post_SC_ = df_cb_uns_pre_SC_cut_.copy()
    df_cb_uns_post_SC_['cluster'] = gmm_cb.predict(df_cb_uns_pre_SC_cut_.values)
    
    df_cb_uns_post_SC_[['log_beta_par_core', 'log_alph_c']] = \
    pow(10., df_cb_uns_post_SC_[['log_beta_par_core', 'log_alph_c']])
    df_cb_uns_post_SC_.rename(columns={"log_beta_par_core": "beta_par_core", "log_alph_c": "alph_c"}, inplace = True)
    
    df_cb_uns_post_SC_['ins_type'] = np.nan
    ins_types_cb_ = ['Oblique FM (B); resonant with Core', 'IC (B); $T_\perp/T_{||} < 1$', 'Oblique FM (B)', \
                    'Oblique Firehose', 'IC (B); $T_\perp/T_{||} > 1$', 'Parallel Firehose', 'IC (B), unstable core', 'IC (C)']
    for cluster, ins_type in zip(np.arange(8), ins_types_cb_):
        df_cb_uns_post_SC_['ins_type'][df_cb_uns_post_SC_['cluster'] == cluster] = ins_type
    
    df_cb_post_SC_ = df_cb_.copy()
    df_cb_post_SC_['ins_type'] = np.nan
    df_cb_post_SC_.loc[df_cb_uns_post_SC_.index, ['ins_type']] = df_cb_uns_post_SC_['ins_type']
    
    return df_cb_post_SC_

### run SAVIC-C - CB 

In [27]:
df_cb_post_SC = SAVIC_C_CB(pd.read_hdf('00_SAVIC_Examples/SAVIC_Examples.h5', key = 'SAVIC-Q_CB_post_Q'))
df_cb_post_SC

,beta_par_core,alph_c,tau_b,alph_b,D_b,vv_b,unstable,group,Pow_core,Pow_beam,kB_angle,ins_type
0,1.0,1.0,1.0,1.0,0.05,0.5,False,NaN,NaN,NaN,NaN,NaN
1,1.5,2.5,0.8,1.0,0.05,0.5,True,3.0,0.193271,0.000000,0.004137,"IC (B), unstable core"
2,0.5,1.0,1.0,3.5,0.10,1.5,True,5.0,0.000000,0.123028,0.003983,IC (B); $T_\perp/T_{||} > 1$
3,0.8,1.1,1.0,1.2,0.05,1.8,True,5.0,0.000000,0.004923,0.001414,IC (B); $T_\perp/T_{||} > 1$
4,0.5,0.7,0.8,0.8,0.01,0.2,False,NaN,NaN,NaN,NaN,NaN
5,0.8,3.1,1.0,3.9,0.10,1.9,True,1.0,0.214777,0.000728,0.000991,"IC (B), unstable core"


##### save results

In [28]:
#df_cb_post_SC.to_hdf('00_SAVIC_Examples/SAVIC_Examples.h5', key = 'SAVIC-C_CB_post_C')

## SAVIC-C - core \& $\alpha$

### load SAVIC-C - C$\alpha$ GM Classifier

In [29]:
gmm_ca_name = 'Output/ML/models/GMM_CA/GMM_CA'
means = np.load(gmm_ca_name + '_means.npy')
covar = np.load(gmm_ca_name + '_covariances.npy')
gmm_ca = GaussianMixture(n_components = len(means), covariance_type='full')
gmm_ca.precisions_cholesky_ = np.linalg.cholesky(np.linalg.inv(covar))
gmm_ca.weights_ = np.load(gmm_ca_name + '_weights.npy')
gmm_ca.means_ = means
gmm_ca.covariances_ = covar

##### generate example data

In [30]:
df_ca = pd.read_hdf('00_SAVIC_Examples/SAVIC_Examples.h5', key = 'SAVIC-Q_CA_post_Q')
df_ca

,beta_par_core,alph_c,tau_a,alph_a,D_a,vv_a,unstable,group,Pow_core,Pow_alpha,kB_angle
0,1.7,0.5,0.5,0.7,0.02,0.07,True,3.0,0.004490,0.000000,0.001962
1,0.6,1.0,0.4,1.9,0.03,0.06,True,5.0,0.000000,0.007504,0.004290
2,0.5,0.7,0.8,0.8,0.01,0.00,False,NaN,NaN,NaN,NaN
3,3.6,0.4,0.6,1.0,0.02,0.15,True,2.0,0.346296,0.000000,0.002381
4,1.0,1.0,0.5,1.2,0.09,0.20,False,NaN,NaN,NaN,NaN
5,0.2,2.4,0.2,0.9,0.03,0.15,True,3.0,0.030773,0.000000,0.002031


In [31]:
df_ca_uns = df_ca[df_ca['unstable']].drop(columns = ['unstable'])
df_ca_uns

,beta_par_core,alph_c,tau_a,alph_a,D_a,vv_a,group,Pow_core,Pow_alpha,kB_angle
0,1.7,0.5,0.5,0.7,0.02,0.07,3.0,0.004490,0.000000,0.001962
1,0.6,1.0,0.4,1.9,0.03,0.06,5.0,0.000000,0.007504,0.004290
3,3.6,0.4,0.6,1.0,0.02,0.15,2.0,0.346296,0.000000,0.002381
5,0.2,2.4,0.2,0.9,0.03,0.15,3.0,0.030773,0.000000,0.002031


##### add instability thresholds

In [32]:
df_ca_uns_pre_SC = df_ca_uns.copy()

df_ca_uns_pre_SC['uns_IC'] = (1. + 0.367 / ( df_ca_uns_pre_SC['beta_par_core'] - 0.011 )**0.364) < df_ca_uns_pre_SC['alph_c']
df_ca_uns_pre_SC['uns_Mirror'] = (1. + 0.702 / ( df_ca_uns_pre_SC['beta_par_core'] + 0.009 )**0.674) < df_ca_uns_pre_SC['alph_c']
df_ca_uns_pre_SC['uns_FM'] = (1. - 0.408 / ( df_ca_uns_pre_SC['beta_par_core'] - 0.410 )**0.529) > df_ca_uns_pre_SC['alph_c']
df_ca_uns_pre_SC['uns_OFH'] = (1. - 1.454 / ( df_ca_uns_pre_SC['beta_par_core'] + 0.178 )**1.023) > df_ca_uns_pre_SC['alph_c']
df_ca_uns_pre_SC['uns_core'] = df_ca_uns_pre_SC['uns_OFH'] | df_ca_uns_pre_SC['uns_FM'] | df_ca_uns_pre_SC['uns_Mirror'] | df_ca_uns_pre_SC['uns_IC']

df_ca_uns_pre_SC['beta_par_alpha'] = df_ca_uns_pre_SC['beta_par_core'] * df_ca_uns_pre_SC['D_a'] / df_ca_uns_pre_SC['tau_a']
df_ca_uns_pre_SC['lambda_FH_beta'] = ( (df_ca_uns_pre_SC['beta_par_core'] + df_ca_uns_pre_SC['beta_par_alpha']) - \
(df_ca_uns_pre_SC['beta_par_core'] * df_ca_uns_pre_SC['alph_c'] + df_ca_uns_pre_SC['beta_par_alpha'] * df_ca_uns_pre_SC['alph_a']) ) / 2.
df_ca_uns_pre_SC['lambda_FH_alpha'] = 4 * df_ca_uns_pre_SC['D_a'] / (1. + df_ca_uns_pre_SC['D_a'] + 4 * df_ca_uns_pre_SC['D_a']) * df_ca_uns_pre_SC['vv_a']**2.
df_ca_uns_pre_SC['lambda_FH'] = df_ca_uns_pre_SC['lambda_FH_beta'] + df_ca_uns_pre_SC['lambda_FH_alpha']
df_ca_uns_pre_SC['lambda_FH'] = df_ca_uns_pre_SC['lambda_FH'] > 1.

df_ca_uns_pre_SC['Pow_core'][df_ca_uns_pre_SC['Pow_core'] < 0] = 0
df_ca_uns_pre_SC['Pow_alpha'][df_ca_uns_pre_SC['Pow_alpha'] < 0] = 0
df_ca_uns_pre_SC

,beta_par_core,alph_c,tau_a,alph_a,D_a,vv_a,group,Pow_core,Pow_alpha,kB_angle,uns_IC,uns_Mirror,uns_FM,uns_OFH,uns_core,beta_par_alpha,lambda_FH_beta,lambda_FH_alpha,lambda_FH
0,1.7,0.5,0.5,0.7,0.02,0.07,3.0,0.004490,0.000000,0.001962,False,False,True,False,True,0.068,0.43520,0.000356,False
1,0.6,1.0,0.4,1.9,0.03,0.06,5.0,0.000000,0.007504,0.004290,False,False,False,False,False,0.045,-0.02025,0.000376,False
3,3.6,0.4,0.6,1.0,0.02,0.15,2.0,0.346296,0.000000,0.002381,False,False,True,True,True,0.120,1.08000,0.001636,True
5,0.2,2.4,0.2,0.9,0.03,0.15,3.0,0.030773,0.000000,0.002031,True,False,False,False,True,0.030,-0.13850,0.002348,False


##### filter and format the input

In [33]:
column_list_ca = ['beta_par_core', 'alph_c', 'kB_angle', \
                  'tau_a', 'D_a', 'alph_a', 'vv_a', 'Pow_core', 'Pow_alpha', \
                  'uns_IC', 'uns_FM', 'uns_OFH', 'uns_core', 'lambda_FH']

df_ca_uns_pre_SC_cut = df_ca_uns_pre_SC[column_list_ca]
df_ca_uns_pre_SC_cut

,beta_par_core,alph_c,kB_angle,tau_a,D_a,alph_a,vv_a,Pow_core,Pow_alpha,uns_IC,uns_FM,uns_OFH,uns_core,lambda_FH
0,1.7,0.5,0.001962,0.5,0.02,0.7,0.07,0.004490,0.000000,False,True,False,True,False
1,0.6,1.0,0.004290,0.4,0.03,1.9,0.06,0.000000,0.007504,False,False,False,False,False
3,3.6,0.4,0.002381,0.6,0.02,1.0,0.15,0.346296,0.000000,False,True,True,True,True
5,0.2,2.4,0.002031,0.2,0.03,0.9,0.15,0.030773,0.000000,True,False,False,True,False


In [34]:
df_ca_uns_pre_SC_cut[['beta_par_core', 'alph_c']] = np.log10(df_ca_uns_pre_SC_cut[['beta_par_core', 'alph_c']])
df_ca_uns_pre_SC_cut.rename(columns={"beta_par_core": "log_beta_par_core", "alph_c": "log_alph_c"}, inplace = True)
df_ca_uns_pre_SC_cut

,log_beta_par_core,log_alph_c,kB_angle,tau_a,D_a,alph_a,vv_a,Pow_core,Pow_alpha,uns_IC,uns_FM,uns_OFH,uns_core,lambda_FH
0,0.230449,-0.301030,0.001962,0.5,0.02,0.7,0.07,0.004490,0.000000,False,True,False,True,False
1,-0.221849,0.000000,0.004290,0.4,0.03,1.9,0.06,0.000000,0.007504,False,False,False,False,False
3,0.556303,-0.397940,0.002381,0.6,0.02,1.0,0.15,0.346296,0.000000,False,True,True,True,True
5,-0.698970,0.380211,0.002031,0.2,0.03,0.9,0.15,0.030773,0.000000,True,False,False,True,False


##### run data through classifier

In [35]:
df_ca_uns_post_SC = df_ca_uns_pre_SC_cut.copy()
df_ca_uns_post_SC['cluster'] = gmm_ca.predict(df_ca_uns_pre_SC_cut.values)
df_ca_uns_post_SC

,log_beta_par_core,log_alph_c,kB_angle,tau_a,D_a,alph_a,vv_a,Pow_core,Pow_alpha,uns_IC,uns_FM,uns_OFH,uns_core,lambda_FH,cluster
0,0.230449,-0.301030,0.001962,0.5,0.02,0.7,0.07,0.004490,0.000000,False,True,False,True,False,0
1,-0.221849,0.000000,0.004290,0.4,0.03,1.9,0.06,0.000000,0.007504,False,False,False,False,False,1
3,0.556303,-0.397940,0.002381,0.6,0.02,1.0,0.15,0.346296,0.000000,False,True,True,True,True,3
5,-0.698970,0.380211,0.002031,0.2,0.03,0.9,0.15,0.030773,0.000000,True,False,False,True,False,5


##### revert to input data values

In [36]:
df_ca_uns_post_SC[['log_beta_par_core', 'log_alph_c']] = pow(10., df_ca_uns_post_SC[['log_beta_par_core', 'log_alph_c']])
df_ca_uns_post_SC.rename(columns={"log_beta_par_core": "beta_par_core", "log_alph_c": "alph_c"}, inplace = True)
df_ca_uns_post_SC

,beta_par_core,alph_c,kB_angle,tau_a,D_a,alph_a,vv_a,Pow_core,Pow_alpha,uns_IC,uns_FM,uns_OFH,uns_core,lambda_FH,cluster
0,1.7,0.5,0.001962,0.5,0.02,0.7,0.07,0.004490,0.000000,False,True,False,True,False,0
1,0.6,1.0,0.004290,0.4,0.03,1.9,0.06,0.000000,0.007504,False,False,False,False,False,1
3,3.6,0.4,0.002381,0.6,0.02,1.0,0.15,0.346296,0.000000,False,True,True,True,True,3
5,0.2,2.4,0.002031,0.2,0.03,0.9,0.15,0.030773,0.000000,True,False,False,True,False,5


##### add instability labels

In [37]:
df_ca_uns_post_SC['ins_type'] = np.nan
ins_types_ca = ['Parallel Firehose', 'IC (A)', 'Oblique Firehose', \
                   'CGL Firehose; Mirror', 'A anis; borderline PFH', 'IC (C)']
for cluster, ins_type in zip(np.arange(6), ins_types_ca):
    df_ca_uns_post_SC['ins_type'][df_ca_uns_post_SC['cluster'] == cluster] = ins_type
df_ca_uns_post_SC

,beta_par_core,alph_c,kB_angle,tau_a,D_a,alph_a,vv_a,Pow_core,Pow_alpha,uns_IC,uns_FM,uns_OFH,uns_core,lambda_FH,cluster,ins_type
0,1.7,0.5,0.001962,0.5,0.02,0.7,0.07,0.004490,0.000000,False,True,False,True,False,0,Parallel Firehose
1,0.6,1.0,0.004290,0.4,0.03,1.9,0.06,0.000000,0.007504,False,False,False,False,False,1,IC (A)
3,3.6,0.4,0.002381,0.6,0.02,1.0,0.15,0.346296,0.000000,False,True,True,True,True,3,CGL Firehose; Mirror
5,0.2,2.4,0.002031,0.2,0.03,0.9,0.15,0.030773,0.000000,True,False,False,True,False,5,IC (C)


##### restore initial input format

In [38]:
df_ca_post_SC = df_ca.copy()
df_ca_post_SC['ins_type'] = np.nan
df_ca_post_SC.loc[df_ca_uns_post_SC.index, ['ins_type']] = df_ca_uns_post_SC['ins_type']
df_ca_post_SC

,beta_par_core,alph_c,tau_a,alph_a,D_a,vv_a,unstable,group,Pow_core,Pow_alpha,kB_angle,ins_type
0,1.7,0.5,0.5,0.7,0.02,0.07,True,3.0,0.004490,0.000000,0.001962,Parallel Firehose
1,0.6,1.0,0.4,1.9,0.03,0.06,True,5.0,0.000000,0.007504,0.004290,IC (A)
2,0.5,0.7,0.8,0.8,0.01,0.00,False,NaN,NaN,NaN,NaN,NaN
3,3.6,0.4,0.6,1.0,0.02,0.15,True,2.0,0.346296,0.000000,0.002381,CGL Firehose; Mirror
4,1.0,1.0,0.5,1.2,0.09,0.20,False,NaN,NaN,NaN,NaN,NaN
5,0.2,2.4,0.2,0.9,0.03,0.15,True,3.0,0.030773,0.000000,0.002031,IC (C)


### load SAVIC-C - C$\alpha$ processing function

In [39]:
def SAVIC_C_CA(df_ca_):
    
    df_ca_uns_ = df_ca_[df_ca_['unstable']].drop(columns = ['unstable'])
    
    df_ca_uns_pre_SC_ = df_ca_uns_.copy()

    df_ca_uns_pre_SC_['uns_IC'] = (1. + 0.367 / ( df_ca_uns_pre_SC_['beta_par_core'] - 0.011 )**0.364) < df_ca_uns_pre_SC_['alph_c']
    df_ca_uns_pre_SC_['uns_Mirror'] = (1. + 0.702 / ( df_ca_uns_pre_SC_['beta_par_core'] + 0.009 )**0.674) < df_ca_uns_pre_SC_['alph_c']
    df_ca_uns_pre_SC_['uns_FM'] = (1. - 0.408 / ( df_ca_uns_pre_SC_['beta_par_core'] - 0.410 )**0.529) > df_ca_uns_pre_SC_['alph_c']
    df_ca_uns_pre_SC_['uns_OFH'] = (1. - 1.454 / ( df_ca_uns_pre_SC_['beta_par_core'] + 0.178 )**1.023) > df_ca_uns_pre_SC_['alph_c']
    df_ca_uns_pre_SC_['uns_core'] = df_ca_uns_pre_SC_['uns_OFH'] | df_ca_uns_pre_SC_['uns_FM'] | df_ca_uns_pre_SC_['uns_Mirror'] | df_ca_uns_pre_SC_['uns_IC']

    df_ca_uns_pre_SC_['beta_par_alpha'] = df_ca_uns_pre_SC_['beta_par_core'] * df_ca_uns_pre_SC_['D_a'] / df_ca_uns_pre_SC_['tau_a']
    df_ca_uns_pre_SC_['lambda_FH_beta'] = ( (df_ca_uns_pre_SC_['beta_par_core'] + df_ca_uns_pre_SC_['beta_par_alpha']) - \
    (df_ca_uns_pre_SC_['beta_par_core'] * df_ca_uns_pre_SC_['alph_c'] + df_ca_uns_pre_SC_['beta_par_alpha'] * df_ca_uns_pre_SC_['alph_a']) ) / 2.
    df_ca_uns_pre_SC_['lambda_FH_alpha'] = 4 * df_ca_uns_pre_SC_['D_a'] / (1. + df_ca_uns_pre_SC_['D_a'] + 4 * df_ca_uns_pre_SC_['D_a']) * df_ca_uns_pre_SC_['vv_a']**2.
    df_ca_uns_pre_SC_['lambda_FH'] = df_ca_uns_pre_SC_['lambda_FH_beta'] + df_ca_uns_pre_SC_['lambda_FH_alpha']
    df_ca_uns_pre_SC_['lambda_FH'] = df_ca_uns_pre_SC_['lambda_FH'] > 1.

    df_ca_uns_pre_SC_['Pow_core'][df_ca_uns_pre_SC_['Pow_core'] < 0] = 0
    df_ca_uns_pre_SC_['Pow_alpha'][df_ca_uns_pre_SC_['Pow_alpha'] < 0] = 0
    
    column_list_ca_ = ['beta_par_core', 'alph_c', 'kB_angle', \
                       'tau_a', 'D_a', 'alph_a', 'vv_a', 'Pow_core', 'Pow_alpha', \
                       'uns_IC', 'uns_FM', 'uns_OFH', 'uns_core', 'lambda_FH']

    df_ca_uns_pre_SC_cut_ = df_ca_uns_pre_SC_[column_list_ca_]
    
    df_ca_uns_pre_SC_cut_[['beta_par_core', 'alph_c']] = np.log10(df_ca_uns_pre_SC_cut_[['beta_par_core', 'alph_c']])
    df_ca_uns_pre_SC_cut_.rename(columns={"beta_par_core": "log_beta_par_core", "alph_c": "log_alph_c"}, inplace = True)
    
    df_ca_uns_post_SC_ = df_ca_uns_pre_SC_cut_.copy()
    df_ca_uns_post_SC_['cluster'] = gmm_ca.predict(df_ca_uns_pre_SC_cut_.values)
    
    df_ca_uns_post_SC_['ins_type'] = np.nan
    ins_types_ca_ = ['Parallel Firehose', 'IC (A)', 'Oblique Firehose', \
                     'CGL Firehose; Mirror', 'A anis; borderline PFH', 'IC (C)']
    for cluster, ins_type in zip(np.arange(6), ins_types_ca_):
        df_ca_uns_post_SC_['ins_type'][df_ca_uns_post_SC_['cluster'] == cluster] = ins_type
    
    df_ca_post_SC_ = df_ca_.copy()
    df_ca_post_SC_['ins_type'] = np.nan
    df_ca_post_SC_.loc[df_ca_uns_post_SC_.index, ['ins_type']] = df_ca_uns_post_SC_['ins_type']
    
    return df_ca_post_SC_

### run SAVIC-C - C$\alpha$ 

In [40]:
df_ca_post_SP = SAVIC_C_CA(pd.read_hdf('00_SAVIC_Examples/SAVIC_Examples.h5', key = 'SAVIC-Q_CA_post_Q'))
df_ca_post_SP

,beta_par_core,alph_c,tau_a,alph_a,D_a,vv_a,unstable,group,Pow_core,Pow_alpha,kB_angle,ins_type
0,1.7,0.5,0.5,0.7,0.02,0.07,True,3.0,0.004490,0.000000,0.001962,Parallel Firehose
1,0.6,1.0,0.4,1.9,0.03,0.06,True,5.0,0.000000,0.007504,0.004290,IC (A)
2,0.5,0.7,0.8,0.8,0.01,0.00,False,NaN,NaN,NaN,NaN,NaN
3,3.6,0.4,0.6,1.0,0.02,0.15,True,2.0,0.346296,0.000000,0.002381,CGL Firehose; Mirror
4,1.0,1.0,0.5,1.2,0.09,0.20,False,NaN,NaN,NaN,NaN,NaN
5,0.2,2.4,0.2,0.9,0.03,0.15,True,3.0,0.030773,0.000000,0.002031,IC (C)


##### save results

In [41]:
#df_ca_post_SP.to_hdf('00_SAVIC_Examples/SAVIC_Examples.h5', key = 'SAVIC-C_CA_post_C')

## SAVIC-C - core, beam \& $\alpha$

### load SAVIC-C - CB$\alpha$ XGB classifier 

In [42]:
gmm_cba_name = 'Output/ML/models/GMM_CBA/GMM_CBA'
means = np.load(gmm_cba_name + '_means.npy')
covar = np.load(gmm_cba_name + '_covariances.npy')
gmm_cba = GaussianMixture(n_components = len(means), covariance_type='full')
gmm_cba.precisions_cholesky_ = np.linalg.cholesky(np.linalg.inv(covar))
gmm_cba.weights_ = np.load(gmm_cba_name + '_weights.npy')
gmm_cba.means_ = means
gmm_cba.covariances_ = covar

##### generate example data

In [43]:
df_cba = pd.read_hdf('00_SAVIC_Examples/SAVIC_Examples.h5', key = 'SAVIC-Q_CBA_post_Q')
df_cba

,beta_par_core,alph_c,tau_b,alph_b,D_b,vv_b,tau_a,alph_a,D_a,vv_a,unstable,group,Pow_core,Pow_beam,Pow_alpha,kB_angle
0,1.7,0.5,1.0,1.0,0.05,0.5,0.5,0.7,0.02,0.07,True,3.0,0.004833,0.000000,0.000000,0.001945
1,0.6,1.0,0.8,1.0,0.05,0.5,0.4,1.9,0.03,0.06,True,7.0,0.000000,0.000000,0.005254,0.004523
2,0.5,0.7,1.0,3.5,0.10,1.5,0.8,0.8,0.01,0.00,True,6.0,0.000000,0.226850,0.000000,0.004767
3,2.2,0.8,1.0,0.7,0.10,0.9,0.6,1.0,0.02,0.15,False,NaN,NaN,NaN,NaN,NaN
4,3.6,0.4,1.2,0.7,0.10,0.8,0.6,1.0,0.03,0.13,True,1.0,0.008698,0.011312,0.000000,0.273413
5,1.0,1.0,1.0,1.2,0.05,1.8,0.5,1.2,0.09,0.20,True,6.0,0.000000,0.012197,0.000000,0.001371
6,0.2,2.4,1.0,3.1,0.10,1.5,0.2,0.9,0.03,0.15,True,3.0,0.035672,0.000000,0.000000,0.001954


In [44]:
df_cba_uns = df_cba[df_cba['unstable']].drop(columns = ['unstable'])
df_cba_uns

,beta_par_core,alph_c,tau_b,alph_b,D_b,vv_b,tau_a,alph_a,D_a,vv_a,group,Pow_core,Pow_beam,Pow_alpha,kB_angle
0,1.7,0.5,1.0,1.0,0.05,0.5,0.5,0.7,0.02,0.07,3.0,0.004833,0.000000,0.000000,0.001945
1,0.6,1.0,0.8,1.0,0.05,0.5,0.4,1.9,0.03,0.06,7.0,0.000000,0.000000,0.005254,0.004523
2,0.5,0.7,1.0,3.5,0.10,1.5,0.8,0.8,0.01,0.00,6.0,0.000000,0.226850,0.000000,0.004767
4,3.6,0.4,1.2,0.7,0.10,0.8,0.6,1.0,0.03,0.13,1.0,0.008698,0.011312,0.000000,0.273413
5,1.0,1.0,1.0,1.2,0.05,1.8,0.5,1.2,0.09,0.20,6.0,0.000000,0.012197,0.000000,0.001371
6,0.2,2.4,1.0,3.1,0.10,1.5,0.2,0.9,0.03,0.15,3.0,0.035672,0.000000,0.000000,0.001954


##### add instability thresholds

In [45]:
df_cba_uns_pre_SC = df_cba_uns.copy()

df_cba_uns_pre_SC['uns_IC'] = (1. + 0.367 / ( df_cba_uns_pre_SC['beta_par_core'] - 0.011 )**0.364) < df_cba_uns_pre_SC['alph_c']
df_cba_uns_pre_SC['uns_Mirror'] = (1. + 0.702 / ( df_cba_uns_pre_SC['beta_par_core'] + 0.009 )**0.674) < df_cba_uns_pre_SC['alph_c']
df_cba_uns_pre_SC['uns_FM'] = (1. - 0.408 / ( df_cba_uns_pre_SC['beta_par_core'] - 0.410 )**0.529) > df_cba_uns_pre_SC['alph_c']
df_cba_uns_pre_SC['uns_OFH'] = (1. - 1.454 / ( df_cba_uns_pre_SC['beta_par_core'] + 0.178 )**1.023) > df_cba_uns_pre_SC['alph_c']
df_cba_uns_pre_SC['uns_core'] = df_cba_uns_pre_SC['uns_OFH'] | df_cba_uns_pre_SC['uns_FM'] | df_cba_uns_pre_SC['uns_Mirror'] | df_cba_uns_pre_SC['uns_IC']


df_cba_uns_pre_SC['beta_par_beam'] = df_cba_uns_pre_SC['beta_par_core'] * df_cba_uns_pre_SC['D_b'] / df_cba_uns_pre_SC['tau_b']
df_cba_uns_pre_SC['beta_par_alpha'] = df_cba_uns_pre_SC['beta_par_core'] * df_cba_uns_pre_SC['D_a'] / df_cba_uns_pre_SC['tau_a']

df_cba_uns_pre_SC['lambda_FH_beta'] = ( (df_cba_uns_pre_SC['beta_par_core'] + df_cba_uns_pre_SC['beta_par_beam'] + df_cba_uns_pre_SC['beta_par_alpha']) - \
(df_cba_uns_pre_SC['beta_par_core'] * df_cba_uns_pre_SC['alph_c'] + df_cba_uns_pre_SC['beta_par_beam'] * df_cba_uns_pre_SC['alph_b'] + \
df_cba_uns_pre_SC['beta_par_alpha'] * df_cba_uns_pre_SC['alph_a']) ) / 2.

df_cba_uns_pre_SC['lambda_FH_alpha'] = 4 * df_cba_uns_pre_SC['D_a'] / (1. + df_cba_uns_pre_SC['D_b'] + 4 * df_cba_uns_pre_SC['D_a']) * df_cba_uns_pre_SC['vv_a']**2.
df_cba_uns_pre_SC['lambda_FH_beam'] = df_cba_uns_pre_SC['D_b'] / (1. + df_cba_uns_pre_SC['D_b'] + 4 * df_cba_uns_pre_SC['D_a']) * df_cba_uns_pre_SC['vv_b']**2.
df_cba_uns_pre_SC['lambda_FH'] = df_cba_uns_pre_SC['lambda_FH_beta'] + df_cba_uns_pre_SC['lambda_FH_alpha'] + df_cba_uns_pre_SC['lambda_FH_beam']
df_cba_uns_pre_SC['lambda_FH'] = df_cba_uns_pre_SC['lambda_FH'] > 1.

df_cba_uns_pre_SC['Pow_core'][df_cba_uns_pre_SC['Pow_core'] < 0] = 0
df_cba_uns_pre_SC['Pow_beam'][df_cba_uns_pre_SC['Pow_beam'] < 0] = 0
df_cba_uns_pre_SC['Pow_alpha'][df_cba_uns_pre_SC['Pow_alpha'] < 0] = 0

df_cba_uns_pre_SC

,beta_par_core,alph_c,tau_b,alph_b,D_b,vv_b,tau_a,alph_a,D_a,vv_a,...,uns_Mirror,uns_FM,uns_OFH,uns_core,beta_par_beam,beta_par_alpha,lambda_FH_beta,lambda_FH_alpha,lambda_FH_beam,lambda_FH
0,1.7,0.5,1.0,1.0,0.05,0.5,0.5,0.7,0.02,0.07,...,False,True,False,True,0.0850,0.06800,0.435200,0.000347,0.011062,False
1,0.6,1.0,0.8,1.0,0.05,0.5,0.4,1.9,0.03,0.06,...,False,False,False,False,0.0375,0.04500,-0.020250,0.000369,0.010684,False
2,0.5,0.7,1.0,3.5,0.10,1.5,0.8,0.8,0.01,0.00,...,False,False,False,False,0.0500,0.00625,0.013125,0.000000,0.197368,False
4,3.6,0.4,1.2,0.7,0.10,0.8,0.6,1.0,0.03,0.13,...,False,True,True,True,0.3000,0.18000,1.125000,0.001662,0.052459,True
5,1.0,1.0,1.0,1.2,0.05,1.8,0.5,1.2,0.09,0.20,...,False,False,False,False,0.0500,0.18000,-0.023000,0.010213,0.114894,False
6,0.2,2.4,1.0,3.1,0.10,1.5,0.2,0.9,0.03,0.15,...,False,False,False,True,0.0200,0.03000,-0.159500,0.002213,0.184426,False


##### filter and format the input

In [46]:
column_list_cba = ['beta_par_core', 'alph_c', 'kB_angle', \
                   'tau_b', 'alph_b', 'D_b', 'vv_b', \
                   'tau_a', 'D_a', 'alph_a', 'vv_a', 'Pow_core', 'Pow_beam', 'Pow_alpha', \
                   'uns_IC', 'uns_FM', 'uns_OFH', 'uns_core', 'lambda_FH']

df_cba_uns_pre_SC_cut = df_cba_uns_pre_SC[column_list_cba]
df_cba_uns_pre_SC_cut

,beta_par_core,alph_c,kB_angle,tau_b,alph_b,D_b,vv_b,tau_a,D_a,alph_a,vv_a,Pow_core,Pow_beam,Pow_alpha,uns_IC,uns_FM,uns_OFH,uns_core,lambda_FH
0,1.7,0.5,0.001945,1.0,1.0,0.05,0.5,0.5,0.02,0.7,0.07,0.004833,0.000000,0.000000,False,True,False,True,False
1,0.6,1.0,0.004523,0.8,1.0,0.05,0.5,0.4,0.03,1.9,0.06,0.000000,0.000000,0.005254,False,False,False,False,False
2,0.5,0.7,0.004767,1.0,3.5,0.10,1.5,0.8,0.01,0.8,0.00,0.000000,0.226850,0.000000,False,False,False,False,False
4,3.6,0.4,0.273413,1.2,0.7,0.10,0.8,0.6,0.03,1.0,0.13,0.008698,0.011312,0.000000,False,True,True,True,True
5,1.0,1.0,0.001371,1.0,1.2,0.05,1.8,0.5,0.09,1.2,0.20,0.000000,0.012197,0.000000,False,False,False,False,False
6,0.2,2.4,0.001954,1.0,3.1,0.10,1.5,0.2,0.03,0.9,0.15,0.035672,0.000000,0.000000,True,False,False,True,False


In [47]:
df_cba_uns_pre_SC_cut[['beta_par_core', 'alph_c']] = np.log10(df_cba_uns_pre_SC_cut[['beta_par_core', 'alph_c']])
df_cba_uns_pre_SC_cut.rename(columns={"beta_par_core": "log_beta_par_core", "alph_c": "log_alph_c"}, inplace = True)
df_cba_uns_pre_SC_cut

,log_beta_par_core,log_alph_c,kB_angle,tau_b,alph_b,D_b,vv_b,tau_a,D_a,alph_a,vv_a,Pow_core,Pow_beam,Pow_alpha,uns_IC,uns_FM,uns_OFH,uns_core,lambda_FH
0,0.230449,-0.301030,0.001945,1.0,1.0,0.05,0.5,0.5,0.02,0.7,0.07,0.004833,0.000000,0.000000,False,True,False,True,False
1,-0.221849,0.000000,0.004523,0.8,1.0,0.05,0.5,0.4,0.03,1.9,0.06,0.000000,0.000000,0.005254,False,False,False,False,False
2,-0.301030,-0.154902,0.004767,1.0,3.5,0.10,1.5,0.8,0.01,0.8,0.00,0.000000,0.226850,0.000000,False,False,False,False,False
4,0.556303,-0.397940,0.273413,1.2,0.7,0.10,0.8,0.6,0.03,1.0,0.13,0.008698,0.011312,0.000000,False,True,True,True,True
5,0.000000,0.000000,0.001371,1.0,1.2,0.05,1.8,0.5,0.09,1.2,0.20,0.000000,0.012197,0.000000,False,False,False,False,False
6,-0.698970,0.380211,0.001954,1.0,3.1,0.10,1.5,0.2,0.03,0.9,0.15,0.035672,0.000000,0.000000,True,False,False,True,False


##### run data through classifier

In [48]:
df_cba_uns_post_SC = df_cba_uns_pre_SC_cut.copy()
df_cba_uns_post_SC['cluster'] = gmm_cba.predict(df_cba_uns_pre_SC_cut.values)
df_cba_uns_post_SC

,log_beta_par_core,log_alph_c,kB_angle,tau_b,alph_b,D_b,vv_b,tau_a,D_a,alph_a,vv_a,Pow_core,Pow_beam,Pow_alpha,uns_IC,uns_FM,uns_OFH,uns_core,lambda_FH,cluster
0,0.230449,-0.301030,0.001945,1.0,1.0,0.05,0.5,0.5,0.02,0.7,0.07,0.004833,0.000000,0.000000,False,True,False,True,False,9
1,-0.221849,0.000000,0.004523,0.8,1.0,0.05,0.5,0.4,0.03,1.9,0.06,0.000000,0.000000,0.005254,False,False,False,False,False,4
2,-0.301030,-0.154902,0.004767,1.0,3.5,0.10,1.5,0.8,0.01,0.8,0.00,0.000000,0.226850,0.000000,False,False,False,False,False,8
4,0.556303,-0.397940,0.273413,1.2,0.7,0.10,0.8,0.6,0.03,1.0,0.13,0.008698,0.011312,0.000000,False,True,True,True,True,0
5,0.000000,0.000000,0.001371,1.0,1.2,0.05,1.8,0.5,0.09,1.2,0.20,0.000000,0.012197,0.000000,False,False,False,False,False,8
6,-0.698970,0.380211,0.001954,1.0,3.1,0.10,1.5,0.2,0.03,0.9,0.15,0.035672,0.000000,0.000000,True,False,False,True,False,7


##### revert to input data values

In [49]:
df_cba_uns_post_SC[['log_beta_par_core', 'log_alph_c']] = pow(10., df_cba_uns_post_SC[['log_beta_par_core', 'log_alph_c']])
df_cba_uns_post_SC.rename(columns={"log_beta_par_core": "beta_par_core", "log_alph_c": "alph_c"}, inplace = True)
df_cba_uns_post_SC

,beta_par_core,alph_c,kB_angle,tau_b,alph_b,D_b,vv_b,tau_a,D_a,alph_a,vv_a,Pow_core,Pow_beam,Pow_alpha,uns_IC,uns_FM,uns_OFH,uns_core,lambda_FH,cluster
0,1.7,0.5,0.001945,1.0,1.0,0.05,0.5,0.5,0.02,0.7,0.07,0.004833,0.000000,0.000000,False,True,False,True,False,9
1,0.6,1.0,0.004523,0.8,1.0,0.05,0.5,0.4,0.03,1.9,0.06,0.000000,0.000000,0.005254,False,False,False,False,False,4
2,0.5,0.7,0.004767,1.0,3.5,0.10,1.5,0.8,0.01,0.8,0.00,0.000000,0.226850,0.000000,False,False,False,False,False,8
4,3.6,0.4,0.273413,1.2,0.7,0.10,0.8,0.6,0.03,1.0,0.13,0.008698,0.011312,0.000000,False,True,True,True,True,0
5,1.0,1.0,0.001371,1.0,1.2,0.05,1.8,0.5,0.09,1.2,0.20,0.000000,0.012197,0.000000,False,False,False,False,False,8
6,0.2,2.4,0.001954,1.0,3.1,0.10,1.5,0.2,0.03,0.9,0.15,0.035672,0.000000,0.000000,True,False,False,True,False,7


##### refined CBA clusters

In [50]:
df_cba_uns_post_SC['r_cluster'] = np.nan
df_cba_uns_post_SC['r_cluster'][df_cba_uns_post_SC['cluster'] == 0] = 9
df_cba_uns_post_SC['r_cluster'][df_cba_uns_post_SC['cluster'] == 1] = 2
df_cba_uns_post_SC['r_cluster'][df_cba_uns_post_SC['cluster'] == 2] = 1
df_cba_uns_post_SC['r_cluster'][df_cba_uns_post_SC['cluster'] == 3] = 4
df_cba_uns_post_SC['r_cluster'][df_cba_uns_post_SC['cluster'] == 4] = 7
df_cba_uns_post_SC['r_cluster'][df_cba_uns_post_SC['cluster'] == 5] = 5
df_cba_uns_post_SC['r_cluster'][df_cba_uns_post_SC['cluster'] == 6] = 12
df_cba_uns_post_SC['r_cluster'][df_cba_uns_post_SC['cluster'] == 7] = 1
df_cba_uns_post_SC['r_cluster'][df_cba_uns_post_SC['cluster'] == 8] = 11
df_cba_uns_post_SC['r_cluster'][df_cba_uns_post_SC['cluster'] == 9] = 10
df_cba_uns_post_SC['r_cluster'][df_cba_uns_post_SC['cluster'] == 10] = 7
df_cba_uns_post_SC['r_cluster'][df_cba_uns_post_SC['cluster'] == 11] = 6
df_cba_uns_post_SC['r_cluster'][df_cba_uns_post_SC['cluster'] == 12] = 8
df_cba_uns_post_SC['r_cluster'][df_cba_uns_post_SC['cluster'] == 13] = 3

df_cba_uns_post_SC['r_cluster'] = df_cba_uns_post_SC['r_cluster'].astype(int)
df_cba_uns_post_SC

,beta_par_core,alph_c,kB_angle,tau_b,alph_b,D_b,vv_b,tau_a,D_a,alph_a,...,Pow_core,Pow_beam,Pow_alpha,uns_IC,uns_FM,uns_OFH,uns_core,lambda_FH,cluster,r_cluster
0,1.7,0.5,0.001945,1.0,1.0,0.05,0.5,0.5,0.02,0.7,...,0.004833,0.000000,0.000000,False,True,False,True,False,9,10
1,0.6,1.0,0.004523,0.8,1.0,0.05,0.5,0.4,0.03,1.9,...,0.000000,0.000000,0.005254,False,False,False,False,False,4,7
2,0.5,0.7,0.004767,1.0,3.5,0.10,1.5,0.8,0.01,0.8,...,0.000000,0.226850,0.000000,False,False,False,False,False,8,11
4,3.6,0.4,0.273413,1.2,0.7,0.10,0.8,0.6,0.03,1.0,...,0.008698,0.011312,0.000000,False,True,True,True,True,0,9
5,1.0,1.0,0.001371,1.0,1.2,0.05,1.8,0.5,0.09,1.2,...,0.000000,0.012197,0.000000,False,False,False,False,False,8,11
6,0.2,2.4,0.001954,1.0,3.1,0.10,1.5,0.2,0.03,0.9,...,0.035672,0.000000,0.000000,True,False,False,True,False,7,1


##### add instability labels

In [51]:
df_cba_uns_post_SC['ins_type'] = np.nan
ins_types_cba = ['IC (C)', 'IC (C); A unstable', \
                 'IC (B), C unstable', 'IC (B); A unstable', 'IC (B); high B anis', 'IC (B); borderline PFH', \
                 'IC (A)', 'IC (A); C absorbing', \
                 'Oblique Firehose', 'Parallel Firehose', \
                 'FM (B), oblique', 'FM (B), oblique; mirror']
for cluster, ins_type in zip(np.arange(12), ins_types_cba):
    df_cba_uns_post_SC['ins_type'][df_cba_uns_post_SC['r_cluster'] == cluster] = ins_type

df_cba_uns_post_SC

,beta_par_core,alph_c,kB_angle,tau_b,alph_b,D_b,vv_b,tau_a,D_a,alph_a,...,Pow_beam,Pow_alpha,uns_IC,uns_FM,uns_OFH,uns_core,lambda_FH,cluster,r_cluster,ins_type
0,1.7,0.5,0.001945,1.0,1.0,0.05,0.5,0.5,0.02,0.7,...,0.000000,0.000000,False,True,False,True,False,9,10,"FM (B), oblique"
1,0.6,1.0,0.004523,0.8,1.0,0.05,0.5,0.4,0.03,1.9,...,0.000000,0.005254,False,False,False,False,False,4,7,IC (A); C absorbing
2,0.5,0.7,0.004767,1.0,3.5,0.10,1.5,0.8,0.01,0.8,...,0.226850,0.000000,False,False,False,False,False,8,11,"FM (B), oblique; mirror"
4,3.6,0.4,0.273413,1.2,0.7,0.10,0.8,0.6,0.03,1.0,...,0.011312,0.000000,False,True,True,True,True,0,9,Parallel Firehose
5,1.0,1.0,0.001371,1.0,1.2,0.05,1.8,0.5,0.09,1.2,...,0.012197,0.000000,False,False,False,False,False,8,11,"FM (B), oblique; mirror"
6,0.2,2.4,0.001954,1.0,3.1,0.10,1.5,0.2,0.03,0.9,...,0.000000,0.000000,True,False,False,True,False,7,1,IC (C); A unstable


##### restore initial input format

In [52]:
df_cba_post_SC = df_cba.copy()
df_cba_post_SC['ins_type'] = np.nan
df_cba_post_SC.loc[df_cba_uns_post_SC.index, ['ins_type']] = df_cba_uns_post_SC['ins_type']
df_cba_post_SC

,beta_par_core,alph_c,tau_b,alph_b,D_b,vv_b,tau_a,alph_a,D_a,vv_a,unstable,group,Pow_core,Pow_beam,Pow_alpha,kB_angle,ins_type
0,1.7,0.5,1.0,1.0,0.05,0.5,0.5,0.7,0.02,0.07,True,3.0,0.004833,0.000000,0.000000,0.001945,"FM (B), oblique"
1,0.6,1.0,0.8,1.0,0.05,0.5,0.4,1.9,0.03,0.06,True,7.0,0.000000,0.000000,0.005254,0.004523,IC (A); C absorbing
2,0.5,0.7,1.0,3.5,0.10,1.5,0.8,0.8,0.01,0.00,True,6.0,0.000000,0.226850,0.000000,0.004767,"FM (B), oblique; mirror"
3,2.2,0.8,1.0,0.7,0.10,0.9,0.6,1.0,0.02,0.15,False,NaN,NaN,NaN,NaN,NaN,NaN
4,3.6,0.4,1.2,0.7,0.10,0.8,0.6,1.0,0.03,0.13,True,1.0,0.008698,0.011312,0.000000,0.273413,Parallel Firehose
5,1.0,1.0,1.0,1.2,0.05,1.8,0.5,1.2,0.09,0.20,True,6.0,0.000000,0.012197,0.000000,0.001371,"FM (B), oblique; mirror"
6,0.2,2.4,1.0,3.1,0.10,1.5,0.2,0.9,0.03,0.15,True,3.0,0.035672,0.000000,0.000000,0.001954,IC (C); A unstable


### load SAVIC-C - CB$\alpha$ processing function

In [53]:
def SAVIC_C_CBA(df_cba_):
    df_cba_uns_ = df_cba_[df_cba_['unstable']].drop(columns = ['unstable'])
    
    df_cba_uns_pre_SC_ = df_cba_uns_.copy()

    df_cba_uns_pre_SC_['uns_IC'] = (1. + 0.367 / ( df_cba_uns_pre_SC_['beta_par_core'] - 0.011 )**0.364) < df_cba_uns_pre_SC_['alph_c']
    df_cba_uns_pre_SC_['uns_Mirror'] = (1. + 0.702 / ( df_cba_uns_pre_SC_['beta_par_core'] + 0.009 )**0.674) < df_cba_uns_pre_SC_['alph_c']
    df_cba_uns_pre_SC_['uns_FM'] = (1. - 0.408 / ( df_cba_uns_pre_SC_['beta_par_core'] - 0.410 )**0.529) > df_cba_uns_pre_SC_['alph_c']
    df_cba_uns_pre_SC_['uns_OFH'] = (1. - 1.454 / ( df_cba_uns_pre_SC_['beta_par_core'] + 0.178 )**1.023) > df_cba_uns_pre_SC_['alph_c']
    df_cba_uns_pre_SC_['uns_core'] = df_cba_uns_pre_SC_['uns_OFH'] | df_cba_uns_pre_SC_['uns_FM'] | df_cba_uns_pre_SC_['uns_Mirror'] | df_cba_uns_pre_SC_['uns_IC']


    df_cba_uns_pre_SC_['beta_par_beam'] = df_cba_uns_pre_SC_['beta_par_core'] * df_cba_uns_pre_SC_['D_b'] / df_cba_uns_pre_SC_['tau_b']
    df_cba_uns_pre_SC_['beta_par_alpha'] = df_cba_uns_pre_SC_['beta_par_core'] * df_cba_uns_pre_SC_['D_a'] / df_cba_uns_pre_SC_['tau_a']

    df_cba_uns_pre_SC_['lambda_FH_beta'] = ( (df_cba_uns_pre_SC_['beta_par_core'] + df_cba_uns_pre_SC_['beta_par_beam'] + df_cba_uns_pre_SC_['beta_par_alpha']) - \
    (df_cba_uns_pre_SC_['beta_par_core'] * df_cba_uns_pre_SC_['alph_c'] + df_cba_uns_pre_SC_['beta_par_beam'] * df_cba_uns_pre_SC_['alph_b'] + \
    df_cba_uns_pre_SC_['beta_par_alpha'] * df_cba_uns_pre_SC_['alph_a']) ) / 2.

    df_cba_uns_pre_SC_['lambda_FH_alpha'] = 4 * df_cba_uns_pre_SC_['D_a'] / (1. + df_cba_uns_pre_SC_['D_b'] + 4 * df_cba_uns_pre_SC_['D_a']) * df_cba_uns_pre_SC_['vv_a']**2.
    df_cba_uns_pre_SC_['lambda_FH_beam'] = df_cba_uns_pre_SC_['D_b'] / (1. + df_cba_uns_pre_SC_['D_b'] + 4 * df_cba_uns_pre_SC_['D_a']) * df_cba_uns_pre_SC_['vv_b']**2.
    df_cba_uns_pre_SC_['lambda_FH'] = df_cba_uns_pre_SC_['lambda_FH_beta'] + df_cba_uns_pre_SC_['lambda_FH_alpha'] + df_cba_uns_pre_SC_['lambda_FH_beam']
    df_cba_uns_pre_SC_['lambda_FH'] = df_cba_uns_pre_SC_['lambda_FH'] > 1.

    df_cba_uns_pre_SC_['Pow_core'][df_cba_uns_pre_SC_['Pow_core'] < 0] = 0
    df_cba_uns_pre_SC_['Pow_beam'][df_cba_uns_pre_SC_['Pow_beam'] < 0] = 0
    df_cba_uns_pre_SC_['Pow_alpha'][df_cba_uns_pre_SC_['Pow_alpha'] < 0] = 0

    column_list_cba_ = ['beta_par_core', 'alph_c', 'kB_angle', \
                        'tau_b', 'alph_b', 'D_b', 'vv_b', \
                        'tau_a', 'D_a', 'alph_a', 'vv_a', 'Pow_core', 'Pow_beam', 'Pow_alpha', \
                        'uns_IC', 'uns_FM', 'uns_OFH', 'uns_core', 'lambda_FH']

    df_cba_uns_pre_SC_cut_ = df_cba_uns_pre_SC_[column_list_cba_]
    
    df_cba_uns_pre_SC_cut_[['beta_par_core', 'alph_c']] = np.log10(df_cba_uns_pre_SC_cut_[['beta_par_core', 'alph_c']])
    df_cba_uns_pre_SC_cut_.rename(columns={"beta_par_core": "log_beta_par_core", "alph_c": "log_alph_c"}, inplace = True)
    
    df_cba_uns_post_SC_ = df_cba_uns_pre_SC_cut_.copy()
    df_cba_uns_post_SC_['cluster'] = gmm_cba.predict(df_cba_uns_pre_SC_cut_.values)
    
    df_cba_uns_post_SC_[['log_beta_par_core', 'log_alph_c']] = \
    pow(10., df_cba_uns_post_SC_[['log_beta_par_core', 'log_alph_c']])
    df_cba_uns_post_SC_.rename(columns={"log_beta_par_core": "beta_par_core", "log_alph_c": "alph_c"}, inplace = True)
    
    df_cba_uns_post_SC_['r_cluster'] = np.nan
    df_cba_uns_post_SC_['r_cluster'][df_cba_uns_post_SC_['cluster'] == 0] = 9
    df_cba_uns_post_SC_['r_cluster'][df_cba_uns_post_SC_['cluster'] == 1] = 2
    df_cba_uns_post_SC_['r_cluster'][df_cba_uns_post_SC_['cluster'] == 2] = 1
    df_cba_uns_post_SC_['r_cluster'][df_cba_uns_post_SC_['cluster'] == 3] = 4
    df_cba_uns_post_SC_['r_cluster'][df_cba_uns_post_SC_['cluster'] == 4] = 7
    df_cba_uns_post_SC_['r_cluster'][df_cba_uns_post_SC_['cluster'] == 5] = 5
    df_cba_uns_post_SC_['r_cluster'][df_cba_uns_post_SC_['cluster'] == 6] = 12
    df_cba_uns_post_SC_['r_cluster'][df_cba_uns_post_SC_['cluster'] == 7] = 1
    df_cba_uns_post_SC_['r_cluster'][df_cba_uns_post_SC_['cluster'] == 8] = 11
    df_cba_uns_post_SC_['r_cluster'][df_cba_uns_post_SC_['cluster'] == 9] = 10
    df_cba_uns_post_SC_['r_cluster'][df_cba_uns_post_SC_['cluster'] == 10] = 7
    df_cba_uns_post_SC_['r_cluster'][df_cba_uns_post_SC_['cluster'] == 11] = 6
    df_cba_uns_post_SC_['r_cluster'][df_cba_uns_post_SC_['cluster'] == 12] = 8
    df_cba_uns_post_SC_['r_cluster'][df_cba_uns_post_SC_['cluster'] == 13] = 3

    df_cba_uns_post_SC_['r_cluster'] = df_cba_uns_post_SC_['r_cluster'].astype(int)
    
    df_cba_uns_post_SC_['ins_type'] = np.nan
    ins_types_cba_ = ['IC (C)', 'IC (C); A unstable', \
                      'IC (B), C unstable', 'IC (B); A unstable', 'IC (B); high B anis', 'IC (B); borderline PFH', \
                      'IC (A)', 'IC (A); C absorbing', \
                      'Oblique Firehose', 'Parallel Firehose', \
                      'FM (B), oblique', 'FM (B), oblique; mirror']
    for cluster, ins_type in zip(np.arange(12), ins_types_cba_):
        df_cba_uns_post_SC_['ins_type'][df_cba_uns_post_SC_['cluster'] == cluster] = ins_type
    
    df_cba_post_SC_ = df_cba_.copy()
    df_cba_post_SC_['ins_type'] = np.nan
    df_cba_post_SC_.loc[df_cba_uns_post_SC_.index, ['ins_type']] = df_cba_uns_post_SC_['ins_type']
    
    return df_cba_post_SC_

### run SAVIC-C - CB$\alpha$ 

In [54]:
df_cba_post_SC = SAVIC_C_CBA(pd.read_hdf('00_SAVIC_Examples/SAVIC_Examples.h5', key = 'SAVIC-Q_CBA_post_Q'))
df_cba_post_SC

,beta_par_core,alph_c,tau_b,alph_b,D_b,vv_b,tau_a,alph_a,D_a,vv_a,unstable,group,Pow_core,Pow_beam,Pow_alpha,kB_angle,ins_type
0,1.7,0.5,1.0,1.0,0.05,0.5,0.5,0.7,0.02,0.07,True,3.0,0.004833,0.000000,0.000000,0.001945,Parallel Firehose
1,0.6,1.0,0.8,1.0,0.05,0.5,0.4,1.9,0.03,0.06,True,7.0,0.000000,0.000000,0.005254,0.004523,IC (B); high B anis
2,0.5,0.7,1.0,3.5,0.10,1.5,0.8,0.8,0.01,0.00,True,6.0,0.000000,0.226850,0.000000,0.004767,Oblique Firehose
3,2.2,0.8,1.0,0.7,0.10,0.9,0.6,1.0,0.02,0.15,False,NaN,NaN,NaN,NaN,NaN,NaN
4,3.6,0.4,1.2,0.7,0.10,0.8,0.6,1.0,0.03,0.13,True,1.0,0.008698,0.011312,0.000000,0.273413,IC (C)
5,1.0,1.0,1.0,1.2,0.05,1.8,0.5,1.2,0.09,0.20,True,6.0,0.000000,0.012197,0.000000,0.001371,Oblique Firehose
6,0.2,2.4,1.0,3.1,0.10,1.5,0.2,0.9,0.03,0.15,True,3.0,0.035672,0.000000,0.000000,0.001954,IC (A); C absorbing


##### save results

In [55]:
#df_cba_post_SC.to_hdf('00_SAVIC_Examples/SAVIC_Examples.h5', key = 'SAVIC-C_CBA_post_C')